In [ ]:
# import requests
# import pandas as pd
# import time

# BASE_URL = "https://api.stackexchange.com/2.3/questions"

# params = {
#     "order": "desc",
#     "sort": "creation",
#     "tagged": "nlp",
#     "site": "stackoverflow",
#     "pagesize": 100,

# }

# # def fetch_questions(pages=200):  # Adjust pages to get 20,000 posts
# #     all_questions = []
# #     for page in range(1, pages + 1):
# #         params["page"] = page
# #         response = requests.get(BASE_URL, params=params)
# #         if response.status_code != 200:
# #             print(f"Error on page {page}: {response.status_code}")
# #             #break
# #         data = response.json()
# #         all_questions.extend(data.get("items", []))
# #         time.sleep(1)  # Avoid rate limiting
# #     return all_questions
# def fetch_questions(pages=200):
#     all_questions = []
#     for page in range(1, pages + 1):
#         params["page"] = page
#         response = requests.get(BASE_URL, params=params)
#         if response.status_code != 200:
#             print(f"Error on page {page}: {response.status_code}. Skipping this page.")
#             continue  # Skip the problematic page
#         data = response.json()
#         all_questions.extend(data.get("items", []))
#         time.sleep(1)  # Avoid rate limiting
#     return all_questions

# questions = fetch_questions()
# df = pd.DataFrame(questions)


Error on page 26: 400. Skipping this page.
Error on page 27: 400. Skipping this page.
Error on page 28: 400. Skipping this page.
Error on page 29: 400. Skipping this page.
Error on page 30: 400. Skipping this page.
Error on page 31: 400. Skipping this page.
Error on page 32: 400. Skipping this page.
Error on page 33: 400. Skipping this page.
Error on page 34: 400. Skipping this page.
Error on page 35: 400. Skipping this page.
Error on page 36: 400. Skipping this page.
Error on page 37: 400. Skipping this page.
Error on page 38: 400. Skipping this page.
Error on page 39: 400. Skipping this page.
Error on page 40: 400. Skipping this page.
Error on page 41: 400. Skipping this page.
Error on page 42: 400. Skipping this page.
Error on page 43: 400. Skipping this page.
Error on page 44: 400. Skipping this page.
Error on page 45: 400. Skipping this page.
Error on page 46: 400. Skipping this page.
Error on page 47: 400. Skipping this page.
Error on page 48: 400. Skipping this page.
Error on pa

In [13]:
import requests
import pandas as pd
from datetime import datetime
import time

def fetch_stackoverflow_posts(tag, page_size=100, max_pages=200):
    base_url = "https://api.stackexchange.com/2.3/questions"
    
    all_questions = []
    
    for page in range(1, max_pages + 1):
        params = {
            'page': page,
            'pagesize': page_size,
            'order': 'desc',
            'sort': 'creation',
            'tagged': tag,
            'site': 'stackoverflow',
            'filter': 'withbody'  # Include question body
        }
        
        response = requests.get(base_url, params=params)
        questions = response.json().get('items', [])
        
        if not questions:
            break
            
        all_questions.extend(questions)
        
        # Respect API rate limits}
        if response.json().get('has_more', False):
            time.sleep(1)  # Wait between requests
        else:
            break
    
    return all_questions



# Fetch NLP questions
nlp_questions = fetch_stackoverflow_posts('nlp')


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
def get_answers_for_question(question_id):
    url = f"https://api.stackexchange.com/2.3/questions/{question_id}/answers"
    params = {
        'order': 'desc',
        'sort': 'votes',
        'site': 'stackoverflow',
        'filter': 'withbody'  # Include answer body
    }
    
    response = requests.get(url, params=params)
    return response.json().get('items', [])


# Create a DataFrame structure
data = []

for question in nlp_questions:
    # Get answers for this question
    answers = get_answers_for_question(question['question_id'])
    
    # Find accepted answer (if any)
    accepted_answer = None
    other_answers = []
    
    for answer in answers:
        if answer.get('is_accepted', False):
            accepted_answer = answer.get('body', '')
        else:
            other_answers.append(answer.get('body', ''))
    
    # Format the data
    question_data = {
        'title': question.get('title', ''),
        'description': question.get('body', ''),
        'tags': question.get('tags', []),
        'accepted_answer': accepted_answer,
        'other_answers': other_answers[:1],  # Just get the first additional answer if exists
        'creation_date': datetime.fromtimestamp(question.get('creation_date', 0)),
        'view_count': question.get('view_count', 0),
        'score': question.get('score', 0)
    }
    
    data.append(question_data)

# Create the DataFrame
df = pd.DataFrame(data)

1000